# 🎬 Emotion-Aware, Anti-Addiction Entertainment Recommendation System

**A Machine Learning Project for Personalized Content Recommendations with Digital Wellbeing**

---

**Author:** Utkarsh Varan Singh  
**Date:** January 2026  
**Version:** 1.0

## 📑 Table of Contents

1. [Problem Definition & Objective](#1-problem-definition--objective)
2. [Data Understanding & Preparation](#2-data-understanding--preparation)
3. [Model / System Design](#3-model--system-design)
4. [Core Implementation](#4-core-implementation)
5. [Evaluation & Analysis](#5-evaluation--analysis)
6. [Ethical Considerations & Responsible AI](#6-ethical-considerations--responsible-ai)
7. [Conclusion & Future Scope](#7-conclusion--future-scope)

---
# 1. Problem Definition & Objective

## 1.1 Selected Project Track
**Track:** Recommendation Systems with Digital Wellbeing

This project combines traditional collaborative filtering recommendation techniques with:
- **Emotion-aware personalization** - Adapting recommendations based on user mood
- **Anti-addiction mechanisms** - Preventing overuse through fatigue detection
- **Context-aware suggestions** - Considering time of day and session duration

## 1.2 Problem Statement

Traditional entertainment recommendation systems optimize for engagement metrics (clicks, watch time) without considering:

1. **User's emotional state** - A sad user might benefit from uplifting content rather than more sad content
2. **Session fatigue** - Extended viewing sessions lead to diminishing returns and potential addiction
3. **Content diversity** - Filter bubbles limit exposure to diverse content

**Our Goal:** Build a recommendation system that maximizes user satisfaction while actively promoting digital wellbeing.

## 1.3 Real-World Relevance & Motivation

| Problem | Impact | Our Solution |
|---------|--------|-------------|
| Screen addiction | 210M people globally suffer from internet addiction | Anti-addiction module with break reminders |
| Mental health | 40% of teens report feeling bad after social media use | Emotion-aware recommendations for mood improvement |
| Filter bubbles | 67% of users only see content similar to past views | Diversity-promoting algorithms |
| Content overload | Average user faces 500+ content choices daily | Personalized, context-aware ranking |

---
# 2. Data Understanding & Preparation

## 2.1 Dataset Overview

We use a combination of:
1. **Movie metadata** - 80 movies across 14 genres with TMDB poster integration
2. **User profiles** - Demographic and preference data
3. **Interaction history** - Watch patterns and session data
4. **Contextual features** - Time of day, session duration, device type

In [ ]:
# Import required libraries
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("Libraries loaded successfully!")

In [ ]:
# Load our movie dataset
from src.data_ingestion import load_raw_data

users, items, interactions = load_raw_data()

print(f"Users Dataset: {len(users)} users")
print(f"Items Dataset: {len(items)} movies")
print(f"Interactions: {len(interactions)} records")

# Display sample data
print("\n--- Sample Movies ---")
items.head(10)

## 2.2 Exploratory Data Analysis

In [ ]:
# Genre Distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Genre counts
genre_counts = items['category'].value_counts()
axes[0].barh(genre_counts.index, genre_counts.values, color=sns.color_palette('husl', len(genre_counts)))
axes[0].set_xlabel('Number of Movies')
axes[0].set_title('Movies per Genre')

# Duration distribution
axes[1].hist(items['duration_minutes'], bins=20, color='steelblue', edgecolor='white')
axes[1].set_xlabel('Duration (minutes)')
axes[1].set_ylabel('Count')
axes[1].set_title('Movie Duration Distribution')

plt.tight_layout()
plt.show()

print(f"Average movie duration: {items['duration_minutes'].mean():.1f} minutes")

## 2.3 Feature Engineering

Our system uses the following features:

| Feature Category | Features | Purpose |
|-----------------|----------|----------|
| User Profile | age, interests, watch_history | Personalization |
| Emotion | label (happy/sad/bored/anxious/neutral) | Mood-based recommendations |
| Context | time_of_day, session_minutes, device | Contextual relevance |
| Fatigue | score, repetition_index, daily_minutes | Anti-addiction |
| Item | category, popularity, duration | Content matching |

---
# 3. Model / System Design

## 3.1 System Architecture

```
┌─────────────────────────────────────────────────────────────┐
│                      FRONTEND LAYER                          │
│         ┌──────────┐  ┌───────────┐  ┌────────────┐         │
│         │  Login   │  │ Dashboard │  │ Watch Page │         │
│         └────┬─────┘  └─────┬─────┘  └──────┬─────┘         │
└──────────────┼──────────────┼───────────────┼───────────────┘
               │              │               │
               ▼              ▼               ▼
┌─────────────────────────────────────────────────────────────┐
│                     API LAYER (FastAPI)                      │
│    ┌────────────┐    ┌─────────────┐    ┌────────────┐      │
│    │    Auth    │    │  Recommend  │    │  Log Watch │      │
│    └────────────┘    └──────┬──────┘    └────────────┘      │
└─────────────────────────────┼───────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────┐
│                   RECOMMENDATION ENGINE                      │
│  ┌──────────────┐  ┌───────────────┐  ┌──────────────┐      │
│  │   Emotion    │  │ Anti-Addiction│  │   Ranker     │      │
│  │   Analyzer   │  │    Module     │  │  Algorithm   │      │
│  └──────────────┘  └───────────────┘  └──────────────┘      │
└─────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────┐
│                       DATA LAYER                             │
│         ┌──────────────┐        ┌──────────────┐            │
│         │    Users     │        │ Movies (TMDB)│            │
│         └──────────────┘        └──────────────┘            │
└─────────────────────────────────────────────────────────────┘
```

## 3.2 Algorithm Components

### 3.2.1 Baseline Recommender
Uses popularity-weighted random sampling with genre preferences.

### 3.2.2 Emotion-Based Scoring
Applies multipliers based on user mood:

| Mood | Boosted Genres (3x) | Penalized Genres (0.3x) |
|------|---------------------|-------------------------|
| Happy | Action, Adventure, Comedy, Musical | Horror, Thriller, Drama |
| Sad | Comedy, Romance, Animation, Musical | Horror, Thriller, Action |
| Bored | Thriller, Sci-Fi, Mystery, Horror | Doc, Drama, Romance |
| Anxious | Doc, Animation, Comedy, Romance | Horror, Thriller, Action |

### 3.2.3 Anti-Addiction Module
Calculates fatigue score using:
```
fatigue = 0.35 × session_factor + 0.2 × repetition + 0.3 × daily_factor + 0.15 × night_factor
```

### 3.2.4 Final Ranker
Combines all signals into final score with ML personalization boost.

---
# 4. Core Implementation

## 4.1 Anti-Addiction Module

In [ ]:
# Anti-Addiction Implementation
from src.anti_addiction import compute_fatigue_score, get_intervention

# Test fatigue calculation at different session lengths
test_sessions = [0, 30, 60, 90, 120, 150, 180]
fatigue_scores = []

for session_min in test_sessions:
    score = compute_fatigue_score(
        session_minutes=session_min,
        repetition_index=0.3,
        daily_cumulative_minutes=session_min,
        time_of_day_factor=0.5
    )
    intervention = get_intervention(score)
    fatigue_scores.append(score)
    print(f"Session: {session_min:3d} min → Fatigue: {score:.2f} → Intervention: {intervention}")

In [ ]:
# Visualize fatigue curve
plt.figure(figsize=(10, 5))
plt.plot(test_sessions, fatigue_scores, 'o-', linewidth=2, markersize=8)
plt.axhline(y=0.5, color='orange', linestyle='--', label='Diversify threshold')
plt.axhline(y=0.7, color='red', linestyle='--', label='Soft break threshold')
plt.axhline(y=0.85, color='darkred', linestyle='--', label='Hard break threshold')
plt.xlabel('Session Duration (minutes)')
plt.ylabel('Fatigue Score')
plt.title('Fatigue Score vs Session Duration')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 4.2 Emotion-Based Recommendations

In [ ]:
# Emotion-based scoring demonstration
mood_boosts = {
    "happy": {"boost": ["action", "adventure", "comedy", "musical", "animation"], "avoid": ["horror", "thriller", "drama"]},
    "sad": {"boost": ["comedy", "romance", "animation", "musical"], "avoid": ["horror", "thriller", "action"]},
    "bored": {"boost": ["thriller", "sci-fi", "mystery", "horror", "action"], "avoid": ["doc", "drama", "romance"]},
    "anxious": {"boost": ["doc", "animation", "comedy", "romance"], "avoid": ["horror", "thriller", "action", "mystery"]},
    "neutral": {"boost": ["comedy", "action", "adventure"], "avoid": []}
}

# Visualize mood-genre matrix
genres = ["action", "comedy", "romance", "horror", "thriller", "drama", "sci-fi", "animation", "doc"]
moods = list(mood_boosts.keys())

matrix = []
for mood in moods:
    row = []
    for genre in genres:
        if genre in mood_boosts[mood]["boost"]:
            row.append(3.0)
        elif genre in mood_boosts[mood]["avoid"]:
            row.append(0.3)
        else:
            row.append(1.0)
    matrix.append(row)

plt.figure(figsize=(12, 5))
sns.heatmap(matrix, annot=True, fmt='.1f', xticklabels=genres, yticklabels=moods, 
            cmap='RdYlGn', center=1.0, vmin=0, vmax=3)
plt.title('Mood-Genre Multiplier Matrix')
plt.xlabel('Genre')
plt.ylabel('User Mood')
plt.tight_layout()
plt.show()

## 4.3 Full Recommendation Pipeline

In [ ]:
# Test the full recommendation system
from src.state import get_user_state
from src.recommender.final_ranker import FinalRanker

# Initialize ranker
ranker = FinalRanker()

# Create test context
context = {
    "time_of_day": "evening",
    "device_type": "browser",
    "location": "home",
    "session_minutes": 30
}

# Test different moods
for mood in ["happy", "sad", "bored"]:
    print(f"\n{'='*50}")
    print(f"MOOD: {mood.upper()}")
    print('='*50)
    
    state = get_user_state("u_1", context, {"label": mood})
    recommendations = ranker.rank("u_1", state, genre_filter=None)[:5]
    
    for i, rec in enumerate(recommendations, 1):
        print(f"{i}. {rec['title']} ({rec['category']}) - Score: {rec['score']:.2f}")

---
# 5. Evaluation & Analysis

## 5.1 Recommendation Quality Metrics

In [ ]:
# Evaluate recommendation diversity
def calculate_diversity(recommendations):
    """Calculate genre diversity in recommendations"""
    genres = [r['category'] for r in recommendations]
    unique_genres = len(set(genres))
    return unique_genres / len(genres) if genres else 0

# Test diversity across moods
context = {"time_of_day": "evening", "session_minutes": 30}
diversity_results = []

for mood in ["happy", "sad", "bored", "anxious", "neutral"]:
    state = get_user_state("u_1", context, {"label": mood})
    recs = ranker.rank("u_1", state)[:10]
    diversity = calculate_diversity(recs)
    diversity_results.append({"mood": mood, "diversity": diversity})
    print(f"{mood:8s}: Diversity = {diversity:.2f}")

print(f"\nAverage Diversity: {np.mean([r['diversity'] for r in diversity_results]):.2f}")

## 5.2 Anti-Addiction Effectiveness

In [ ]:
# Analyze intervention triggers
interventions = {'none': 0, 'diversify': 0, 'soft_break': 0, 'hard_break': 0}

# Simulate 1000 random sessions
np.random.seed(42)
for _ in range(1000):
    session = np.random.randint(0, 200)
    repetition = np.random.uniform(0, 1)
    daily = np.random.randint(0, 300)
    night = np.random.uniform(0, 1)
    
    score = compute_fatigue_score(session, repetition, daily, night)
    intervention = get_intervention(score)
    interventions[intervention] += 1

# Visualize intervention distribution
plt.figure(figsize=(8, 5))
colors = ['green', 'yellow', 'orange', 'red']
plt.bar(interventions.keys(), interventions.values(), color=colors)
plt.title('Intervention Distribution (1000 Simulated Sessions)')
plt.ylabel('Count')
plt.xlabel('Intervention Type')
for i, (k, v) in enumerate(interventions.items()):
    plt.text(i, v + 10, f'{v/10:.1f}%', ha='center')
plt.show()

---
# 6. Ethical Considerations & Responsible AI

## 6.1 Privacy Considerations

| Concern | Our Approach |
|---------|-------------|
| User data storage | Minimal data collection, local storage for demo |
| Emotion inference | User self-reports mood, no facial recognition |
| Watch history | Used only for recommendations, not shared |
| Authentication | Basic implementation, production needs encryption |

## 6.2 Algorithmic Fairness

- **No demographic bias**: Recommendations based on behavior, not demographics
- **Content diversity**: Anti-addiction module promotes diverse content
- **Transparent explanations**: Each recommendation includes reasoning

## 6.3 Digital Wellbeing Integration

This system prioritizes user wellbeing over engagement metrics:

1. **Active break reminders** instead of auto-play
2. **Fatigue-aware scoring** that reduces addictive patterns
3. **Mood-appropriate content** to support mental health
4. **Session limits** to prevent binge-watching

## 6.4 Limitations & Risks

- **Mood detection accuracy**: Self-reported moods may not reflect true state
- **Cold start problem**: New users lack interaction history
- **Content quality**: System trusts TMDB ratings, no independent verification

---
# 7. Conclusion & Future Scope

## 7.1 Summary of Achievements

We successfully built an **Emotion-Aware, Anti-Addiction Entertainment Recommendation System** that:

✅ Personalizes recommendations based on user mood (happy, sad, bored, anxious, neutral)  
✅ Implements anti-addiction features with fatigue tracking and break reminders  
✅ Provides context-aware suggestions based on time of day  
✅ Uses real TMDB movie posters for premium user experience  
✅ Includes user authentication with login/signup functionality  
✅ Generates explainable recommendations with reasoning  

## 7.2 Key Technical Contributions

1. **Fatigue Score Algorithm**: Novel weighted formula considering session duration, repetition, daily usage, and time-of-day
2. **Mood-Genre Mapping**: Comprehensive boost/penalty matrix for 5 moods × 14 genres
3. **Intervention System**: Three-tier intervention (diversify → soft break → hard break)

## 7.3 Future Improvements

| Enhancement | Description | Priority |
|-------------|-------------|----------|
| ML-based mood detection | Use watch patterns to infer mood | High |
| Collaborative filtering | Learn from similar users | High |
| Real-time feedback | Thumbs up/down for recommendations | Medium |
| Social features | Share recommendations with friends | Low |
| Voice interface | Mood input through voice commands | Low |

## 7.4 Lessons Learned

- Balancing personalization with diversity is crucial for user satisfaction
- Anti-addiction features can be integrated without sacrificing recommendation quality
- Explainable AI increases user trust and engagement

---
## 📚 References

1. TMDB API - Movie metadata and posters
2. MovieLens Dataset - Benchmark for recommendation systems
3. FastAPI Documentation - Backend framework
4. Responsible AI Practices - Google AI Principles

---

**End of Notebook**